<a href="https://colab.research.google.com/github/Karasiari/Graphs/blob/main/Clearing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "git+https://github.com/Karasiari/HuGraphFinal.git"

  Cloning https://github.com/Karasiari/HuGraphFinal.git to /tmp/pip-req-build-4__v_m63
  Running command git clone --filter=blob:none --quiet https://github.com/Karasiari/HuGraphFinal.git /tmp/pip-req-build-4__v_m63
  Resolved https://github.com/Karasiari/HuGraphFinal.git to commit 3bb6abd51ee12dcbfc74f9bacc124cf9f53645f2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hugraphfin: filename=hugraphfin-0.1.0-py3-none-any.whl size=32387 sha256=5203259e70aa31f2e50f5d163c311ceade35e6cab83dd30732adfabcee00f795
  Stored in directory: /tmp/pip-ephem-wheel-cache-l90_bt35/wheels/46/83/89/572a6db8e6320eb877bf6ced236fa61f993538a12c5e43c72e
Successfully built hugraphfin


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import os
from google.colab import files

# две функции - читалки hu графов в мультиграфы

BITRATE_DENOMINATOR = 100

def hu_csv_to_graphs(path, demands_path, capacity_path):
  Network = pd.read_csv(path, header=None, names = ['id', 'source', 'target', 'length'])
  Traffic = pd.read_csv(demands_path, header=None, names = ['id', 'source', 'target', 'bitrate'])
  Capacity = pd.read_csv(capacity_path, header=None, names = ['name', 'value'])
  network = Network.iloc[1:].copy()
  traffic = Traffic.iloc[1:].copy()
  capacity = Capacity.iloc[1:].copy()

  network['source'] = network['source'].astype(int)
  network['target'] = network['target'].astype(int)
  traffic['source'] = traffic['source'].astype(int)
  traffic['target'] = traffic['target'].astype(int)
  traffic['bitrate'] = traffic['bitrate'].astype(float) / BITRATE_DENOMINATOR
  capacity_value = float(capacity[capacity['name']=='NumberOfWavelengths']['value'])

  sources = network['source'].tolist()
  targets = network['target'].tolist()
  sources_traffic = traffic['source'].tolist()
  targets_traffic = traffic['target'].tolist()
  bitrates_traffic = traffic['bitrate'].tolist()

  unique_vertices = set()
  for source, target in zip(sources, targets):
    unique_vertices.add(source)
    unique_vertices.add(target)
  vertex_mapping = {old: new for new, old in enumerate(unique_vertices)}
  num_vertices = len(unique_vertices)

  adj_edges = []
  traffic_edges = []

  for source, target in zip(sources, targets):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    adj_edges.append((new_source, new_target, {"capacity": capacity_value}))
  for source, target, bitrate in zip(sources_traffic, targets_traffic, bitrates_traffic):
    new_source = vertex_mapping[source]
    new_target = vertex_mapping[target]
    traffic_edges.append((new_source, new_target, {"weight": bitrate}))

  adj_graph = nx.MultiGraph()
  adj_graph.add_edges_from(adj_edges)
  traffic_graph = nx.MultiDiGraph()
  traffic_graph.add_nodes_from(range(adj_graph.number_of_nodes()))
  traffic_graph.add_edges_from(traffic_edges)

  return adj_graph, traffic_graph, capacity_value

def get_hu_graphs(base_path, specified_graphs, specified):
  Graphs = {}
  csv_tables = []
  for folder_name in os.listdir(base_path):
    if folder_name in specified_graphs or not specified:
      folder_path = os.path.join(base_path, folder_name)
      csv_path = os.path.join(folder_path, 'links.csv')
      csv_path_demands = os.path.join(folder_path, 'demands.csv')
      csv_path_capacity = os.path.join(folder_path, 'params.csv')
      csv_tables.append(folder_name)

      adj_graph, traffic_graph, capacity_value = hu_csv_to_graphs(csv_path, csv_path_demands, csv_path_capacity)
      Graphs[folder_name] = {'adj_graph': adj_graph, 'traffic_graph': traffic_graph, 'capacity_value': capacity_value}

  return Graphs

path_to_folder = '/content/drive/MyDrive/Кола'

In [3]:
hu_graphs_names = ['cola_t8']
hu_graphs = get_hu_graphs(path_to_folder, hu_graphs_names, True)

/tmp/ipython-input-3554038479.py:24: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  capacity_value = float(capacity[capacity['name']=='NumberOfWavelengths']['value'])


In [ ]:
from graph_exps.exp import *
available_volumes = ((1, 0.9), (2, 0.1))

for name, graph in hu_graphs.items():
  Graph = HuGraphForExps(graph['adj_graph'], graph['traffic_graph'])
  additional_resources = [80.0, 80.0, 80.0]
  allocation_types = ['alpha', 'random_alpha', 'random']
  results = expand_test_for_graph(Graph, additional_resources, allocation_types, tries_for_allocation=10, epsilon=1.2, available_volumes=available_volumes)

In [5]:
results

allocation solved rerouted volume ratio  \
alpha        1               True                  1.02   
             2               True                  1.02   
             3               True                  1.02   
             4               True                  1.02   
             5               True                  1.02   
             6               True                  1.02   
             7               True                  1.02   
             8               True                  1.02   
             9               True                  1.02   
             10              True                  1.02   
random_alpha 1               True                  1.02   
             2               True                  1.02   
             3               True                  1.01   
             4               True                  1.02   
             5               True                  1.01   
             6               True                  1.02   
             7               True                  1.01   
             8               True                  1.02   
             9               True                  1.02   
             10              True                  1.02   
random       1               True                  1.01   
             2               True                  1.01   
             3               True                  1.02   
             4               True                  1.02   
             5               True                  1.02   
             6               True                  1.02   
             7               True                  1.02   
             8               True                  1.02   
             9               True                  1.02   
             10              True                  1.02   

                gamma for failed (0, 1) gamma for failed (0, 3)  \
alpha        1                      inf                    38.0   
             2                      inf                    38.0   
             3                      inf                    38.0   
             4                      inf                    38.0   
             5                      inf                    38.0   
             6                      inf                    38.0   
             7                      inf                    38.0   
             8                      inf                    38.0   
             9                      inf                    38.0   
             10                     inf                    38.0   
random_alpha 1                      inf                    38.0   
             2                      inf                    38.0   
             3                      inf                    38.0   
             4                      inf                    38.0   
             5                      inf                    38.0   
             6                      inf                    38.0   
             7                      inf                    38.0   
             8                      inf                    38.0   
             9                      inf                    38.0   
             10                     inf                    38.0   
random       1                      inf                    38.0   
             2                      inf                    38.0   
             3                      inf                    38.0   
             4                      inf                    38.0   
             5                      inf                    38.0   
             6                      inf                    38.0   
             7                      inf                    38.0   
             8                      inf                    38.0   
             9                      inf                    38.0   
             10                     inf                    38.0   

                gamma for failed (0, 19) gamma for failed (0, 21)  \
alpha        1                     57.7